<a href="https://colab.research.google.com/github/Paulagilv/Prediccion-del-tiempo-de-permanencia-de-pacientes-en-un-hospital/blob/main/02_Preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random as rnd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.linear_model import LinearRegression

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_validate, ShuffleSplit




from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error,mean_squared_log_error


from sklearn.model_selection import ShuffleSplit, KFold,cross_val_score

In [ ]:
# Obtencion de los archivos en formato Zip desde repositorio Github
!wget https://github.com/anishoond/Archivos_Proyecto/blob/main/Archivos_proyecto.zip?raw=true

!unzip /content/Archivos_proyecto.zip?raw=true

--2023-05-28 03:48:16--  https://github.com/anishoond/Archivos_Proyecto/blob/main/Archivos_proyecto.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/anishoond/Archivos_Proyecto/raw/main/Archivos_proyecto.zip [following]
--2023-05-28 03:48:16--  https://github.com/anishoond/Archivos_Proyecto/raw/main/Archivos_proyecto.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/anishoond/Archivos_Proyecto/main/Archivos_proyecto.zip [following]
--2023-05-28 03:48:17--  https://raw.githubusercontent.com/anishoond/Archivos_Proyecto/main/Archivos_proyecto.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|18

In [ ]:
data_org =pd.read_csv("train.csv",index_col=0)

DT_in2 = pd.read_csv("train.csv",index_col=0)

In [ ]:
def random_fill_nan(col_name_nan, DTf_rep_nan):
    
    """col_name_nan es el nombre de la columna en donde se tienen valores 
    NaN
    
    DTf_rep_nan es el data frame que contiene la columna con nombre col_name_nan
    y en donde se van a reemplzar los valores por NaN por otro valor.
    
    Los datos NaN se reemplzan por un valor aleatorio contenido en los valores
    existentes en las columnas.
    
    La función retorna un El data set ingresado sin datos NaN
    """
    
    
    val_na_col = DTf_rep_nan[col_name_nan].isna()
    
    indices_nan= val_na_col.loc[val_na_col == True].index
    
    values_in_nan_cols = np.unique(DTf_rep_nan[col_name_nan])

    min_val_replace = min(values_in_nan_cols)
    
    max_val_replace = max(values_in_nan_cols)
    
    for indx_nan_r in indices_nan:
       
        new_nan1 = float(rnd.randint(min_val_replace,max_val_replace))
        
        DTf_rep_nan.loc[indx_nan_r , col_name_nan] = new_nan1
    
    return 'done'


In [ ]:
def frecuencia_per_data(DATA, NAMES):
    
    """DATA = data set a modificar
    NAMES =  nombre de columna categorica
    
    devuelve un data set con solo números, pues se reemplazan las variables
    categoricas por su frecuencia de aparición"""

    col_a_contar = DATA[NAMES]
    
    val_con_ele = np.unique(col_a_contar)
    
    fre_per_name = np.array([np.sum(col_a_contar==val_in_col) for val_in_col in val_con_ele])
    
    
    index_per_val_in_col = np.array([np.where(col_a_contar==val_index) for val_index in val_con_ele ])
    
    
    for i in range(len(fre_per_name)):
        DATA[NAMES].iloc[index_per_val_in_col[i][0]] = fre_per_name[i]
    
    
    DATA_FRE = DATA[:]
    return DATA_FRE

In [ ]:
def rangos_edades (rango_str, DTf_ages, col_name_org,new_col_max):
    """rango_str =  es un string con dos valores, un minimo y un maximo
    
    DTf_ages, es un data frame en donde se agregaran los valores minimos
    y maximos extraidos de rango_str a la columna correspondiente
    
    como resultado de la función se tendrá que en el data frame entregado 
    se escribirán los valores minimos y maximos en las columnas especificadas

    """
    
    guion = rango_str.index('-')
    
    min_rango = int(rango_str[0:guion])
    
    max_rango = int(rango_str[guion+1:])
    
    
    
    
    DTf_ages.loc[DTf_ages[col_name_org] == rango_str, new_col_max] = max_rango
    
    return min_rango, max_rango

In [ ]:
"""Lo primero que se hace es eliminar los datos
nan del data set, random_fill_nan en cada una de las columnas
que contenga datos nan"""
col_names_nan = []

for na_obj in DT_in2.columns:
    
    if np.sum(DT_in2[na_obj].isna()) !=0:
        col_names_nan.append(na_obj)

for c_nan_name in col_names_nan:
  random_fill_nan(c_nan_name,DT_in2)

In [ ]:
"""Una vez el data set tenga datos no nan, se procede 
a unificar el tipo de datos, es decir se crea un data set con datos
numericos unicamente. Se aplica la función frecuencia_per_data en cada una de las
columnas que presenten variables categoricas"""

C_STR_names = []

for i in DT_in2.columns:
    if DT_in2[i].dtype == 'object':
     C_STR_names.append(i) 

for cc in C_STR_names[:len(C_STR_names)-1]:
    frecuencia_per_data(DT_in2,cc)

for age in np.unique(DT_in2['Stay']):
    try:
        rangos_edades(age,DT_in2,'Stay','Stay_max')
    except:
        continue

DT_in2.drop('Stay', axis = 'columns', inplace=True)

DT_in2['Stay_max'].iloc[np.where(DT_in2['Stay_max'].isna())] = 101

<ipython-input-5-182279463347>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  index_per_val_in_col = np.array([np.where(col_a_contar==val_index) for val_index in val_con_ele ])
<ipython-input-5-182279463347>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA[NAMES].iloc[index_per_val_in_col[i][0]] = fre_per_name[i]
<ipython-input-8-d73dc3163b0d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [ ]:
DT_in2

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay_max
case_id,,,,,,,,,,,,,,,,,
1,8,45928,3,62674,3,28516,127947,112753,2.0,31397,7.0,117676,56723,2,48514,4911.0,10.0
2,2,45928,5,62674,2,28516,77794,112753,2.0,31397,7.0,152261,56723,2,48514,5954.0,50.0
3,10,24770,1,133336,2,29649,77794,55351,2.0,31397,7.0,152261,56723,2,48514,4745.0,40.0
4,26,68946,2,122428,2,28516,127947,51809,2.0,31397,7.0,152261,56723,2,48514,7272.0,50.0
5,26,68946,2,122428,2,28516,77794,51809,2.0,31397,7.0,152261,56723,2,48514,5558.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318434,6,143425,6,133336,3,28516,106165,112753,4.0,86499,23.0,117676,175843,3,63749,4144.0,20.0
318435,24,143425,1,133336,2,29649,106165,55351,4.0,325,8.0,48501,175843,4,7890,6699.0,40.0
318436,7,143425,4,133336,3,249486,127947,112753,4.0,125235,10.0,117676,85872,3,35792,4235.0,20.0
